In [1]:
import requests
import urllib.request
import json
import Levenshtein
import requests
import os
import sys
sys.path.append('../')

from tqdm import tqdm
from statistics import mean
from SPARQLWrapper import SPARQLWrapper, JSON

from Identification_couples_livres.extract_books_from_DB import *

## Wikidata

### Recherche des auteurs dans Wikidata

Les classes potentiellement pertinentes dans Wikidata sont les suivantes:

* wd:Q36180 Écrivain
    * wd:Q49757	    poète
    * wd:Q214917	dramaturge
    * wd:Q381353	femme de lettres
    * wd:Q6625963	romancier
    * wd:Q4853732	auteur de littérature pour la jeunesse
    * wd:Q5434338	écrivain de fantasy
    * wd:Q1626130	homme de lettres
    * wd:Q10297252	auteur de roman policier
    * wd:Q18844224	écrivain de science-fiction
    * wd:Q15980158	essayiste
    * wd:Q26203955	auteur de contes
    * wd:Q11774202  essayiste
    * wd:Q1930187   jounraliste
    * wd:Q487596    dramaturge
    * wd:Q201788    historien


Dans une expérience préliminaire nous avons tenté de retrouver les personnes qui sont des instances de ces classes, nées au Canada et dont la langue est le français.

Si on prend la liste des écrivains qu'on a ainsi extraite, et qu'on analyse les écrivains  qui ne se retrouvent pas dans DBpdedia, on peut faire les constatations suivantes:

* Près du quart sont effectivement des écrivains québécois (les autres sont en général des auteurs hors Québec, des universitaires, ou des gens qui ne sont tout simplement pas écrivains)
* Parmi les écrivains québécois, leur absence de la liste peut être expliquée par les raisons suivantes:
    * Erreur dans l'écriture du nom, faisant en sorte que le nom ne correspond pas à l'URI dans DBpedia (Marc-André au lieu de Marc André, par exemple)
    * La personne est présente dans Wikipedia mais pas dans DBpedia
    * Dans la plupart des cas, les catégories associées à la personne n'ont pas permis de l'identifier clairement comme écrivain
* Parmi les écrivains québécois, vraiment très peu sont membres de l'UNEQ (ce qui signifie que cela ne nous aiderait pas à les détecter)


Il serait risqué d'utiliser cette liste extraite de Wikidata, puisqu'on aura environ 4000 instances qui ne sont pas des écrivains québécois.

On va donc plutôt faire le contraire: on va tenter de prendre les écrivains que nous avons extrait de DBpedia et les retrouver dans Wikidata.

In [ ]:
from time import sleep

f_dbpedia_fr = open('../Data/DBpedia/ecrivains_dbpedia_fr.txt')
f_dbpedia_en = open('../Data/DBpedia/ecrivains_dbpedia_fr.txt')

ecrivains = set()
for l in f_dbpedia_fr.readlines():
    (_,nom) = l.strip().split(' ; ')
    ecrivains.add(nom)
f_dbpedia_fr.close()

for l in f_dbpedia_en.readlines():
    (_,nom) = l.strip().split(' ; ')
    ecrivains.add(nom)
f_dbpedia_en.close()


q_ecrivain = """
SELECT  DISTINCT ?s 
WHERE 
{{

  {{ ?s rdfs:label "{0}"@fr }} UNION {{ ?s rdfs:label "{0}"@en }} UNION {{ ?s skos:altLabel "{0}"@fr }}
  
  {{ ?s wdt:P106  wd:Q36180 }} UNION    
  {{ ?s wdt:P106  wd:Q49757 }} UNION
  {{ ?s wdt:P106  wd:Q214917 }} UNION
  {{ ?s wdt:P106  wd:Q381353 }} UNION
  {{ ?s wdt:P106  wd:Q6625963 }} UNION
  {{ ?s wdt:P106  wd:Q4853732 }} UNION
  {{ ?s wdt:P106  wd:Q5434338 }} UNION
  {{ ?s wdt:P106  wd:Q1626130 }} UNION
  {{ ?s wdt:P106  wd:Q10297252 }} UNION 
  {{ ?s wdt:P106  wd:Q18844224 }} UNION 
  {{ ?s wdt:P106  wd:Q15980158 }} UNION 
  {{ ?s wdt:P106  wd:Q26203955 }} UNION
  {{ ?s wdt:P106  wd:Q11774202 }} UNION
  {{ ?s wdt:P106  wd:Q1930187 }} UNION
  {{ ?s wdt:P106  wd:Q487596 }} UNION
  {{ ?s wdt:P106  wd:Q201788 }}

}}
"""

ecrivains_wikidata = []

for e in ecrivains:
    sleep(1)
    print("---" + e + "----")
    rep_wikidata_ecrivain = requests.get('https://query.wikidata.org/sparql?format=json&query=' + q_ecrivain.format(e))
    if rep_wikidata_ecrivain.status_code == 200:
        if len(rep_wikidata_ecrivain.json()['results']['bindings']) > 0:
            for resultat in rep_wikidata_ecrivain.json()['results']['bindings']:
                ecrivains_wikidata.append((e, resultat['s']['value']))
                print(resultat['s'])
        else:
            print("PAS TROUVÉ DANS WIKIDATA")
    else:
        print('ERREUR:',e)


In [ ]:
print("Sur les",len(ecrivains), 'trouvés dans DBpedia, on en retrouve',len(ecrivains_wikidata), "dans Wikidata")

In [ ]:
f = open('../Data/Wikidata/ecrivains_wikidata2.txt','w')
for (nom,uri) in ecrivains_wikidata:
    f.write("{}<>{}\n".format(nom,uri))
f.close()

## Recherche des auteurs des bases de données dans wikidata

### Numéros de propriété importantes:

- sex or gender (P21)
- country of citizenship (P27)
- name in native language (P1559)
- birth name (P1477)
-  given name  (P735)
- family name (P734)
- date of birth (P569)
- place of birth (P19)
- date of death  (P570)
- place of death  (P20)
- occupation (P106)
    - author (Q482980)
    - writer (Q36180)
- notable work (P800)
- genre (P136)
- award received (P166)
- nominated for (P1411)
- country of citizenship (P27)

On recherche par nom, et par occupation (occupation fille de "ecrivant"), et on en récupère les labels

In [3]:
q = """
SELECT DISTINCT ?s ?book ?bookLabel
WHERE
{{
    ?s wdt:P31 wd:Q5 .
    ?s wdt:P106 ?occup .
    ?occup wdt:P279 wd:Q36180 .

    {{ ?s rdfs:label "{0}"@fr }} UNION {{ ?s rdfs:label "{0}"@en }} UNION {{ ?s skos:altLabel "{0}"@fr }} .
    OPTIONAL {{ ?s wdt:P800 ?book . }}

    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "fr,en".}}
}}
"""

On test avec les acteurs connus

In [4]:
famous = ['Michel Tremblay', 'Anne Hébert', 'Gabrielle Roy', 'Marie Cardinal', 'Réjean Ducharme',
                'Jacques Ferron', 'Victor-Lévy Beaulieu', 'Marcel Dubé', 'Yves Thériault', 'Jacques Poulin',
                'André Langevin']

search_results = {}
print('tset query')
nb_author_found = 0
for author in tqdm(famous, total=len(famous)):
    time.sleep(1)
    rep_wikidata_ecrivain = requests.get('https://query.wikidata.org/sparql?format=json&query=' + q.format(author))

    if rep_wikidata_ecrivain.status_code == 429:
        print("trop rapide !!!")

    if rep_wikidata_ecrivain.status_code == 200:
        if len(rep_wikidata_ecrivain.json()['results']['bindings']) > 0:
            nb_author_found += 1
            search_results[author] = {'id':rep_wikidata_ecrivain.json()['results']['bindings'][0]['s']['value'], 'books':[]}
            for result in rep_wikidata_ecrivain.json()['results']['bindings']:
                try:
                    search_results[author]['books'].append({'book_id': result['book']['value'], 'book_label': result['bookLabel']['value']})
                except KeyError:
                    pass
print("Nombre d'auteurs trouvés: ", nb_author_found, " soit ", nb_author_found * 100/len(famous), "%")

100%|██████████| 11/11 [00:15<00:00,  1.36s/it]


tset query
Nombre d'auteurs trouvés:  11  soit  100.0 %


On peut essayer avec tous les auteurs de la base de donnée mais ca prendrais beaucoup de temps

In [5]:
if os.path.isfile('../Data/raw_authors_ls.json'):
    with open('../Data/raw_authors_ls.json', 'r') as input_file:
        raw_author_ls = json.load(input_file, cls=BookJSONDecoder)
else:
    all_books = generate_all_books()
    author_ls, raw_author_ls = generate_all_authors(all_books)


search_results = {}
print('query')
nb_author_found = 0
for author in tqdm(raw_author_ls, total=len(raw_author_ls)):
    time.sleep(1)
    rep_wikidata_ecrivain = requests.get('https://query.wikidata.org/sparql?format=json&query=' + q.format(author))

    if rep_wikidata_ecrivain.status_code == 429:
        print("trop rapide !!!")

    if rep_wikidata_ecrivain.status_code == 200:
        if len(rep_wikidata_ecrivain.json()['results']['bindings']) > 0:
            nb_author_found += 1
            search_results[author] = {'id':rep_wikidata_ecrivain.json()['results']['bindings'][0]['s']['value'], 'books':[]}
            for result in rep_wikidata_ecrivain.json()['results']['bindings']:
                try:
                    search_results[author]['books'].append({'book_id': result['book']['value'], 'book_label': result['bookLabel']['value']})
                except KeyError:
                    pass
print("Nombre d'auteurs trouvés: ", nb_author_found, " soit ", nb_author_found * 100/len(raw_author_ls), "%")

with open('../Data/Wikidata/wikidata_author_&_books_found_by_name_search.json', 'w') as outfile:
    json.dump(search_results, outfile)

  0%|          | 12/26126 [00:14<8:50:14,  1.22s/it]


query


KeyboardInterrupt: 

## Recherche de tout les auteurs, puis comparaison avec notre bases de donnée manuellement

### Recherche de tout les auteurs
Première query: ecrivains canadiens parlant/ecrivant en francais => 1232 resultats

In [6]:
q = """
SELECT ?item ?itemLabel
WHERE
{{

    ?item wdt:P31 wd:Q5 .
    ?item wdt:P106 wd:Q36180 .
    ?item wdt:P27 wd:Q16 .
    ?item wdt:P1412 wd:Q150

    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".}
}}"""

Deuxième query: ecrivain canadiens parlant/ecrivant en francais qui ont un produit notable => 82 resultats

In [ ]:
q = """
SELECT ?item ?itemLabel ?bookLabel
WHERE
{

  ?item wdt:P31 wd:Q5 .
  ?item wdt:P106 wd:Q36180 .
  ?item wdt:P27 wd:Q16 .
  ?item wdt:P1412 wd:Q150 .
  ?item wdt:P800 ?book

  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".}
}"""

Dernière query: auteurs francophones, leures oeuvres et prix

In [7]:
all_authors_query = """
SELECT DISTINCT ?author ?authorLabel ?bookLabel ?awardLabel
WHERE
{
    ?author wdt:P31 wd:Q5 .
    ?author wdt:P27 ?country .
    ?country wdt:P2936 wd:Q150 .
    ?author wdt:P106 ?occup .
    ?occup wdt:P279 wd:Q36180

    OPTIONAL { ?author wdt:P800 ?book . }
    OPTIONAL { ?author wdt:P166 ?award . }

    SERVICE wikibase:label { bd:serviceParam wikibase:language "fr,en".}

}"""
rep_wikidata_ecrivain = requests.get('https://query.wikidata.org/sparql?format=json&query=' + all_authors_query)
print('code_status: ', rep_wikidata_ecrivain.status_code)
if rep_wikidata_ecrivain.status_code == 200:
    if len(rep_wikidata_ecrivain.json()['results']['bindings']) > 0:
        print('success ! Nombre d\e lignes: ', len(rep_wikidata_ecrivain.json()['results']['bindings']))

code_status:  200
success ! Nombre d\e lignes:  25726


# On sauvegarde le résultat de la recherche car elle peut échouer en fonction de la charge sur les serveurs de wikidata

In [8]:
if rep_wikidata_ecrivain.status_code == 200:
    if len(rep_wikidata_ecrivain.json()['results']['bindings']) > 0:
        with open('../Data/Wikidata/request_allfrench_author_books_price_wikidata.json', 'w') as output_file:
            json.dump(rep_wikidata_ecrivain.json()['results']['bindings'], output_file)

On formate les infromation sous forme de dictionnaire

In [12]:
with open('../Data/Wikidata/request_allfrench_author_books_price_wikidata.json') as input_file:
    res = json.load(input_file)

cleaned_data = {}

for result in res:
    try:
        cleaned_data[result['author']['value']]['works'].add(result['bookLabel']['value'])
        cleaned_data[result['author']['value']]['awards'].add(result['awardLabel']['value'])
    except KeyError:
        cleaned_data[result['author']['value']] = {
            'author_uri': result['author']['value'],
            'author_label': result['authorLabel']['value'],
            'works': {[result['bookLabel']['value']]} if 'bookLable' in result else set(),
            'awards': {result['awardLabel']['value']} if 'awardLabel' in result else set()
        }

with open('../Data/Wikidata/parsed_request_allfrench_author_books_price_wikidata.json', 'w') as output_file:
    json.dump(cleaned_data, output_file, cls=BookJSONEncoder)

### Comparaison avec nos bases de données

In [13]:
with open('../Data/Wikidata/parsed_request_allfrench_author_books_price_wikidata.json', 'r') as author_file:
    wikidata_authors = json.load(author_file)

if os.path.isfile('../Data/authors_ls.json'):
    with open('../Data/authors_ls.json', 'r') as input_file:
        author_ls = json.load(input_file, cls=BookJSONDecoder)
else:
    all_books = generate_all_books()
    author_ls, raw_author_ls = generate_all_authors(all_books)

# Couple dont on estime que les deux auteurs sont les mêmes
couple_writers = []
# Couple dont on estime que les deux auteurs sont différents mais à la limite d'être les mêmes
neg_couple_writers = []

# Compte les livres des auteurs par base de donnée (permet de voir si une base de donnée n'a aucun match)
data_base_stats = {}
# Compte les auteurs de wikidata qui ont été trouvé dans nos bases de donnée
found_at_least_once = 0
# Compte le nombre de couples
nb_couple = 0
# Compte le nombre de fois qu'un auteur de wikidata a été trouvé dans nos bases de donnée
author_found_freq = {}

for wiki_author_id, wiki_author_info in tqdm(wikidata_authors.items()):
    normalised_wikidata_title = normalize(wiki_author_info['author_label'])
    # Compte le nombre de fois que l'auteur wikidata a été associé a un auteur de nos bases de donnée
    wiki_author_found_count = 0
    for DB_author, DB_author_books in author_ls.items():
        dist_auteur = Levenshtein.distance(DB_author, normalised_wikidata_title)
        if max(1, min(len(DB_author), len(normalised_wikidata_title)) / 6) > dist_auteur > max(1, min(len(DB_author), len(normalised_wikidata_title)) / 8):
            neg_couple_writers.append({
                        'author_DB': {'name': DB_author, 'books': DB_author_books},
                        'author_wiki': {'name': normalised_wikidata_title,
                                        'name_raw': wiki_author_info['author_label'],
                                        'info': wiki_author_info}
                        })
        elif dist_auteur < max(1, min(len(DB_author), len(normalised_wikidata_title)) / 8):

            new_couple = {
                        'author_DB': {'name': DB_author, 'books': DB_author_books},
                        'author_wiki': {'name': normalised_wikidata_title,
                                        'name_raw': wiki_author_info['author_label'],
                                        'info': wiki_author_info}
                    }

            nb_couple += 1
            wiki_author_found_count += 1

            couple_writers.append(new_couple)
            for book in DB_author_books:
                try:
                    data_base_stats[book.data_base] += 1
                except:
                    data_base_stats[book.data_base] = 1

    if wiki_author_found_count:
        found_at_least_once += 1
        author_found_freq[wiki_author_info['author_label']] = wiki_author_found_count

print("nombre d'autheurs en commun: ", found_at_least_once, " soit ", found_at_least_once * 100 / len(list(author_ls)), "%")
print("nombre d'autheurs reconnus: ", nb_couple, " soit ", nb_couple/found_at_least_once, ' auteur dans nos bases de donnée par auteur reconnu dans wikidata')
print("Frequence de reconnaissance des 10 auteurs de wikidata les plus reconnus: ", sorted(author_found_freq.items(), key=lambda item: item[1], reverse=True)[:10])


print("Répartition des livres des auteurs reconnus dans les bases de données: ")
print('1/8: ', json.dumps(data_base_stats))

print("20 premiers couples à la limite d'être considérés comme égaux")
for couple in neg_couple_writers[:20]:
    print("Autheur DB: ", couple['author_DB']['name'] + " " * (40 - len(couple['author_DB']['name'])),
          "Autheur wikidata: ", couple['author_wiki']['name'])

with open('../Data/Wikidata/couple_authors_wikidata.json', 'w') as outfile:
    json.dump(couple_writers, outfile, cls=BookJSONEncoder)

100%|██████████| 18846/18846 [07:56<00:00, 39.52it/s]


nombre d'autheurs en commun:  1301  soit  6.353159488231273 %
nombre d'autheurs reconnus:  1378  soit  1.059185242121445  auteur dans nos bases de donnée par auteur reconnu dans wikidata
Frequence de reconnaissance des 10 auteurs de wikidata les plus reconnus:  [('Michel Côté', 3), ('Denise Boucher', 3), ('Marie José Thériault', 3), ('Michel Tremblay', 3), ('Sylvie Bérard', 3), ('Jean-Claude Lauzon', 2), ('Gaëtan Brulotte', 2), ('Anne Hébert', 2), ('Michel Marc Bouchard', 2), ('Denis Thériault', 2)]
Répartition des livres des auteurs reconnus dans les bases de données: 
1/8:  {"ILE": 13478, "ADP": 2681, "Babelio": 2172, "Depot_legal": 8765, "Hurtubise": 334}
20 premiers couples à la limite d'être considérés comme égaux
Autheur DB:  david descoteaux                         Autheur wikidata:  david decoteau
Autheur DB:  pierre lambert                           Autheur wikidata:  pierre aubert
Autheur DB:  pierre hebert                            Autheur wikidata:  pierre aubert
Autheur D

On s'intéresse aux information récoltées sur ces auteurs

In [16]:
count_books = 0
nb_books_for_each_wiki_author = []

count_awards = 0
nb_awards_for_each_wiki_author = []

all_awards_name = set()

for couple in couple_writers:
    if couple['author_wiki']['info']['works']:
        count_books += 1

    if couple['author_wiki']['info']['awards']:
        count_awards += 1
        all_awards_name.update(couple['author_wiki']['info']['awards'])

    nb_books_for_each_wiki_author.append(len(couple['author_wiki']['info']['works']))
    nb_awards_for_each_wiki_author.append(len(couple['author_wiki']['info']['awards']))
print("Nombre d'artiste ayant des livres attribués: ", count_books)
print("Nombre de livres global récupérés: ", sum(nb_books_for_each_wiki_author))

print("Nombre d'artiste ayant des prix attribués: ", count_awards)
print("Nombre de prix global récupérés: ", sum(nb_awards_for_each_wiki_author))

print("exemple des prix: ", list(all_awards_name)[:20])

Nombre d'artiste ayant des livres attribués:  50
Nombre de livres global récupérés:  137
Nombre d'artiste ayant des prix attribués:  422
Nombre de prix global récupérés:  683
exemple des prix:  ['prix littéraire PEN Oakland/Josephine Miles', "grand officier de l'ordre national du Québec", "membre de l'Académie royale des arts du Canada", 'Aurealis Award for best science fiction novel', "grand officier de la Légion d'honneur", 'Canadian Library Association Young Adult Book Award', 'prix Nils-Holgersson', 'prix du scénario du Festival de Cannes', "Commonwealth Writers' Prize", 'Prix Robert-Cliche', "docteur honoris causa de l'université de Liège", 'Prix Jeunesse des univers parallèles', "Ours d'argent de la meilleure actrice", 'prix Dan-David', 'Mårbacka Award', 'prix Femina', "Grammy Award de l'album de l'année", 'prix Goncourt de la poésie', 'Pour le Mérite pour les sciences et arts', 'Ethel Wilson Fiction Prize']


On veut confirmer nos couples d'auteur a partir des titres des livres qu'ils ont ecrits
On va comparer les titres uns à uns.
- Si un des titre ou plus correspond, alors on confirmera ce couple,
- Si aucune titre n'est en commun, on considera le couple comme faux ou incomplet
- Si on n'a pas récupéré de titre depuis wikidata alors on ne peut pas conclure

In [19]:
with open('../Data/Wikidata/couple_authors_wikidata.json', 'r') as input_file:
    couple_writers = json.load(input_file, cls=BookJSONDecoder)

# Listes pour stoker nos couples confirmé, invalidé et ceux dont on ne peut pas conclure car il n'y a pas de titres
couples_confirmed = []
couples_infirmed = []
couples_unsure = []
# Compte les cas ci-dessus
count_checked_by_titles = 0
count_no_titles = 0
count_differentiate_by_titles = 0

for couple in tqdm(couple_writers):
    confirmed_by_title_match = False
    confirmed_by_title_match_re = False

    # On vérifie si on peut trouver des titres proches en comparant les titres parsés
    # au titres normalisés de nos bases de donnée
    book_titles_DB = [book.title for book in couple['author_DB']['books']]
    for title in book_titles_DB:
        normalised_wikidata_title_ls = [normalize(title_wikidata) for title_wikidata in couple['author_wiki']['info']['works']]
        for title_wikidata in normalised_wikidata_title_ls:
            dist_titles = Levenshtein.distance(title, title_wikidata)
            if dist_titles < max(1, min(len(title), len(title_wikidata)) / 4):
                confirmed_by_title_match = True
                break
    # On recherche par le suite le titre du livre directement dans les titres non nettoyé de la page wikipédia
    # via expression régulières
    for title in book_titles_DB:
        for title_wiki_raw in couple['author_wiki']['info']['works']:
            if re.search(normalize(title), normalize(title_wiki_raw)):
                confirmed_by_title_match_re = True
                break

    # On liste les cas possibles: 0,1,2 tests réussis
    if (not confirmed_by_title_match) and (not confirmed_by_title_match_re):
        if couple['author_wiki']['info']['works']:
            count_differentiate_by_titles += 1
            couples_infirmed.append(couple)
        else:
            count_no_titles += 1
            couples_unsure.append(couple)
    else:
        count_checked_by_titles += 1
        couples_confirmed.append(couple)

    # On sauvegarde comment avons nous trouvé les correspondances entre les deux listes de titres
    couple['confirmed_by_title_match'] = confirmed_by_title_match
    couple['confirmed_by_title_match_re'] = confirmed_by_title_match_re

print('nombre de couple: ', len(couple_writers))
print('nombre de couples confirmés par titres: ', count_checked_by_titles)
print('nombre de couples infirmés par titres: ', count_differentiate_by_titles)
print('nombre de couples sans titres de wikipedia: ', count_no_titles)

with open('../Data/Wikidata/couple_authors_wikidata_confirmed.json', 'w') as outfile:
    json.dump(couples_confirmed, outfile, cls=BookJSONEncoder)

with open('../Data/Wikidata/couple_authors_wikidata_infirmed.json', 'w') as outfile:
    json.dump(couples_infirmed, outfile, cls=BookJSONEncoder)

with open('../Data/Wikidata/couple_authors_wikidata_unsure.json', 'w') as outfile:
    json.dump(couples_unsure, outfile, cls=BookJSONEncoder)

100%|██████████| 1378/1378 [00:00<00:00, 10303.25it/s]


nombre de couple:  1378
nombre de couples confirmés par titres:  23
nombre de couples infirmés par titres:  27
nombre de couples sans titres de wikipedia:  1328


## Extraction  des champs disponibles depuis DBpédia pour les auteurs en commun

### Récupération de tout les champs

Recherche de tout les champs disponibles depuis wikidata soit depuis tout les auteurs francophones,
soit depuis les auteurs dont on a trouvé une instance dans notre base de donnée

#### Recherche des URI de tout les auteurs de wikidata francophones

In [20]:
all_authors_query = """
SELECT DISTINCT ?author
WHERE
{
    ?author wdt:P31 wd:Q5 .
    ?author wdt:P27 ?country .
    ?country wdt:P2936 wd:Q150 .
    ?author wdt:P106 ?occup .
    ?occup wdt:P279 wd:Q36180

}"""
rep_wikidata_ecrivain = requests.get('https://query.wikidata.org/sparql?format=json&query=' + all_authors_query)
print('code_status: ', rep_wikidata_ecrivain.status_code)

authors_uri = []
if rep_wikidata_ecrivain.status_code == 200:
    if len(rep_wikidata_ecrivain.json()['results']['bindings']) > 0:
        print('exemples: ')
        print(json.dumps(rep_wikidata_ecrivain.json()['results']['bindings'][:2], indent=2))
        for result in tqdm(rep_wikidata_ecrivain.json()['results']['bindings']):
            authors_uri.append(result['author']['value'])

100%|██████████| 18846/18846 [00:00<00:00, 653914.62it/s]


code_status:  200
exemples: 
[
  {
    "author": {
      "type": "uri",
      "value": "http://www.wikidata.org/entity/Q32335"
    }
  },
  {
    "author": {
      "type": "uri",
      "value": "http://www.wikidata.org/entity/Q33801"
    }
  }
]


#### Seletion des champs:

In [22]:
with open('../Data/Wikidata/couple_authors_wikidata.json', 'r') as input_file:
    couple_writers = json.load(input_file, cls=BookJSONDecoder)

result = {}
# # Version pour tout les auteurs francophones
# for uri in tqdm(authors_uri):
# Version pour tout les auteurs dont on a trouvé un équivalent dans notre base de donnée
for couple in tqdm(couple_writers):
    uri = couple['author_wiki']['info']['author_uri'].split('/')[-1]
    # partie générique
    time.sleep(1)
    get_all_fields_query = """
    SELECT ?pLabel ?objLabel
    WHERE {{
        wd:{0} ?predicate ?obj.
        BIND( IRI(REPLACE( STR(?predicate),"prop/direct/","entity/" )) AS ?p)
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "fr, en". }}
    }}
    """
    rep_wikidata_ecrivain = requests.get('https://query.wikidata.org/sparql?format=json&query=' + get_all_fields_query.format(uri))

    if rep_wikidata_ecrivain.status_code == 429:
            print("trop rapide !!!")

    if rep_wikidata_ecrivain.status_code == 200:
        if len(rep_wikidata_ecrivain.json()['results']['bindings']) > 0:

            result[uri] = [(duo['pLabel']['value'], duo['objLabel']['value']) for duo in rep_wikidata_ecrivain.json()['results']['bindings']]
    else:
        print("erreur code: ", rep_wikidata_ecrivain.status_code)

100%|██████████| 1378/1378 [32:58<00:00,  1.44s/it] 


erreur code:  504


#### Tri et sauvegarde des champs

In [23]:
# Toutes les propriété possibles
all_properties = set()
for properties in result.values():
    all_properties.update([property[0]for property in properties])

# Pour chaque auteur, on compte combien de fois chaque propriété apparait
properties_dict = {}
for author_uri, properties in result.items():
    for knowed_property in all_properties:
        counter = 0
        for property in properties:
            if property[0] == knowed_property:
                counter += 1
        try:
            properties_dict[knowed_property].append(counter)
        except:
            properties_dict[knowed_property] = [counter]

# On tire des mesures du dictionnaire créé si desssus
properties_resume = {}
for knowed_property in properties_dict:
    properties_resume[knowed_property] = {
        # Moyenne du nombre de fois qu'apparait une propriété quand elle est présente chez un auteur
        'non_null_mean': mean([e for e in properties_dict[knowed_property] if e]),
        # Nombres d'auteur qui ont cette propriété
        'nb_non_null': sum([1 for e in properties_dict[knowed_property] if e]),
        # Nombre total de fois qu'apparait une propriété
        'sum': sum(properties_dict[knowed_property]),
    }

# On sauvegarde ces mesures sous csv
with open('../Data/Wikidata/list_properies_by_author.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['name', 'non_null_mean', 'nb_non_null', 'sum' ])
        writer.writeheader()
        for rep_prop_name, rep_prop_dict in properties_resume.items():
            if rep_prop_name.find("http://www.wikidata.org/prop/") == -1:
                rep_prop_dict['name'] = rep_prop_name
                writer.writerow(rep_prop_dict)
